In [ ]:
import json
from requests_sse import EventSource
import pandas as pd
import time

url = 'https://stream.wikimedia.org/v2/stream/mediawiki.recentchange'

# Adding headers can help in case the server requires specific request formatting
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.83 Safari/537.36'
}

# Function to determine if the change is to a talk page
def is_talk_page(title):
    # Typically, talk pages start with "Talk:" or "<Language> talk:"
    # This will handle "Talk:", "User talk:", "Wikipedia talk:", etc.
    return any(title.lower().startswith(prefix) for prefix in ['talk:', 'user talk:', 'wikipedia talk:', 'file talk:', 
                                                              'template talk:', 'help talk:', 'category talk:', 'portal talk:',
                                                              'book talk:', 'draft talk:', 'timedtext talk:', 'module talk:'])
list= [[],[],[]]
t_end = time.time() + 40
# Setting up the EventSource connection
with EventSource(url, headers=headers) as stream:
    for event in stream:
        if time.time() > t_end:
            break

        if event.type == 'message':
            try:
                # Parse the event data as JSON
                change = json.loads(event.data)
                # Check if the change is related to a talk page from Wikipedia
                if change['wiki'].endswith('wiki') and is_talk_page(change['title']) and change['bot'] == False and change['wiki']=='enwiki':
                    print('{user} edited {title} on {wiki}: {comment}'.format(
                        user=change['user'], title=change['title'], wiki=change['wiki'], comment = change['comment']))
                    list[0].append(change['title'])
                    list[1].append(change['user'])
                    list[2].append(change['comment'])
                    data = {
                        "Title": list[0],
                        "User": list[1],
                        "Comment": list[2]
                    }
            
            except ValueError:
                # In case of any issues in parsing JSON data
                continue

df = pd.DataFrame(data)
print(df)




71.163.151.156 edited Talk:Stephen Mitchell (translator) on enwiki: /* To Mr. Mitchell */ Reply
LuniZunie edited User talk:Arstotskiano on enwiki: Welcome to Wikipedia! ([[WP:AntiVandal|AV]])
XLinkBot edited User talk:Flapjack06 on enwiki: BOT - Notifying user of reverted link additions to [[Cumberland Law Review]] (good faith remark) - 1/0/0 - links: web.archive.org/web/20250914101850/blurblawg.typepad.com/files/harper-lee-symposium.pdf, blurblawg.typepad.com/files/harper-lee-symposium.pdf
